In [ ]:
import sys
sys.path.insert(0, '..')
import qtm.qcompilation
import matplotlib.pyplot as plt
import qiskit
import qtm.qsp
import qtm.ansatz, qtm.state
%load_ext autoreload
%autoreload 2

: 

In [ ]:
def f(num_qubits, num_layers, anszat, vdagger_input):
    optimizer = 'adam'

    if anszat == 'g2':
        u_input = qtm.ansatz.g2(num_qubits, num_layers) 
        ansatz_input = qtm.ansatz.g2
    if anszat == 'g2gn':
        u_input = qtm.ansatz.g2gn(num_qubits, num_layers) 
        ansatz_input = qtm.ansatz.g2gn
    if anszat == 'g2gnw':
        u_input = qtm.ansatz.g2gnw(num_qubits, num_layers) 
        ansatz_input = qtm.ansatz.g2gnw
        
    if vdagger_input == 'ghz':
            vdagger_input = qtm.state.create_ghz_state(num_qubits).inverse()
    if vdagger_input == 'W':
            vdagger_input = qtm.state.create_W_state(num_qubits).inverse()
    if vdagger_input == 'AME':
            vdagger_input = qtm.state.create_AME_state(num_qubits).inverse()   
    if vdagger_input == 'haar':
            vdagger_input = qtm.state.create_haar_state(num_qubits).inverse() 
    
    compiler = qtm.qcompilation.QuantumCompilation(
        u = u_input,
        vdagger = vdagger_input,
        optimizer = optimizer,
        loss_func = 'loss_fubini_study'
    )
    compiler.fit(num_steps = 1, verbose = 1)
    qspobj = qtm.qsp.QuantumStatePreparation.load_from_compiler(
    compiler = compiler, ansatz = ansatz_input)
    qspobj.save(state = vdagger_input, file_name='../experiments/qsp/')

    return 

# Tests
f(3,1, 'g2', 'AME')

: 

In [ ]:
if __name__ == "__main__":
    list_num_qubits = [3,4,5,6,7,8,9,10]
    list_num_layers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    list_u = ['g2', 'g2gn', 'g2gnw']
    list_v = ['ghz', 'W', 'AME', 'haar']

    i = len(list_num_qubits)
    j = len(list_num_layers)
    k = len(list_u)
    l = len(list_v)

    processes = [[[[0 for _ in range(l)] for _ in range(k)] for _ in range(j)] for _ in range(i)]
    
    for i in range(len(list_num_qubits)):
        for j in range(len(list_num_layers)):
            for k in range(len(list_u)):
                for l in range(len(list_v)):
                    processes[i][j][k][l] = (multiprocessing.Process(target = f, args=(list_num_qubits[i], list_num_layers[j], list_u[k], list_v[l])))

    for i in range(len(list_num_qubits)):
        for j in range(len(list_num_layers)):
            for k in range(len(list_u)):
                for l in range(len(list_v)):
                    processes[i][j][k][l].start()

    for i in range(len(list_num_qubits)):
        for j in range(len(list_num_layers)):
            for k in range(len(list_u)):
                for l in range(len(list_v)):
                    processes[i][j][k][l].join()

    print("Done!")

: 